# Setup MCP Toolbox

MCP Toolbox for Databases is an open source MCP server that enables you to develop Database tools for agents easier, faster, and more securely by handling the complexities such as connection pooling, authentication, and more. Toolbox supports [16+ database engines](https://googleapis.github.io/genai-toolbox/resources/sources/), including the Spanner and AlloyDB instances we provisioned for this lab.

Toolbox sits between your application’s orchestration framework and your database, providing a control plane that is used to modify, distribute, or invoke tools. It simplifies the management of your tools by providing you with a centralized location to store and update tools, allowing you to share tools between agents and applications and update those tools without necessarily redeploying your application.

![MCP Toolbox Architecture](img/toolbox_architecture.png)

This notebook walks you through creating two simple MCP Toolbox tools that can access data in Spanner and AlloyDB, deploying them securely to Cloud Run, and manually executing the tools.

## Basic Setup

### Define Notebook Variables

Update the variables below to match your environment. You will be prompted for the AlloyDB password you chose then you provisioned the environment with Terraform.

In [ ]:
# Project variables
project_id = "my-project"
region = "my-region"
vpc = "demo-vpc"
gcs_bucket_name = f"project-files-{project_id}"

# AlloyDB variables
alloydb_cluster = "my-alloydb-cluster"
alloydb_instance = "my-alloydb-instance"
alloydb_database = "finance"
alloydb_password = input("Please enter the password for the AlloyDB 'postgres' database user: ")

# Spanner variables
spanner_instance = "my-spanner-instance"
spanner_database = "finance-graph"
session = None

In [ ]:
# Set env variable to suppress annoying system warnings when running shell commands
%env GRPC_ENABLE_FORK_SUPPORT=1

### Connect to your Google Cloud Project

In [ ]:
# Configure gcloud.
!gcloud config set project {project_id}

### Configure Logging

In [ ]:
import logging
import sys

# Configure the root logger to output messages with INFO level or above
logging.basicConfig(level=logging.INFO, stream=sys.stdout, format='%(asctime)s[%(levelname)5s][%(name)14s] - %(message)s',  datefmt='%H:%M:%S', force=True)

### Install Dependencies

In [ ]:
! pip install --quiet google-cloud-storage==2.19.0 \
                      toolbox-core==0.2.1


### Define Helper Functions

#### REST API Helper Function

In [ ]:
import requests
import google.auth
import json

# Get an access token based upon the current user
creds, _ = google.auth.default()
authed_session = google.auth.transport.requests.AuthorizedSession(creds)
access_token=creds.token

if project_id:
  authed_session.headers.update({"x-goog-user-project": project_id}) # Required to workaround a project quota bug

def rest_api_helper(
    url: str,
    http_verb: str,
    request_body: dict = None,
    params: dict = None,
    session: requests.Session = authed_session,
  ) -> dict:
  """Calls a REST API using a pre-authenticated requests Session."""

  headers = {"Content-Type": "application/json"}

  try:

    if http_verb == "GET":
      response = session.get(url, headers=headers, params=params)
    elif http_verb == "POST":
      response = session.post(url, json=request_body, headers=headers, params=params)
    elif http_verb == "PUT":
      response = session.put(url, json=request_body, headers=headers, params=params)
    elif http_verb == "PATCH":
      response = session.patch(url, json=request_body, headers=headers, params=params)
    elif http_verb == "DELETE":
      response = session.delete(url, headers=headers, params=params)
    else:
      raise ValueError(f"Unknown HTTP verb: {http_verb}")

    # Raise an exception for bad status codes (4xx or 5xx)
    response.raise_for_status()

    # Check if response has content before trying to parse JSON
    if response.content:
        return response.json()
    else:
        return {} # Return empty dict for empty responses (like 204 No Content)

  except requests.exceptions.RequestException as e:
      # Catch potential requests library errors (network, timeout, etc.)
      # Log detailed error information
      print(f"Request failed: {e}")
      if e.response is not None:
          print(f"Request URL: {e.request.url}")
          print(f"Request Headers: {e.request.headers}")
          print(f"Request Body: {e.request.body}")
          print(f"Response Status: {e.response.status_code}")
          print(f"Response Text: {e.response.text}")
          # Re-raise a more specific error or a custom one
          raise RuntimeError(f"API call failed with status {e.response.status_code}: {e.response.text}") from e
      else:
          raise RuntimeError(f"API call failed: {e}") from e
  except json.JSONDecodeError as e:
      print(f"Failed to decode JSON response: {e}")
      print(f"Response Text: {response.text}")
      raise RuntimeError(f"Invalid JSON received from API: {response.text}") from e


## Setup MCP Toolbox

### Define `tools.yaml`

MCP Toolbox tools are defined in a tools.yaml file, using the structure defined below:

- `Sources` represent your different data sources that a tool can interact with. 
- `Tools` define actions an agent can take – such as reading and writing to a source. Typically, a tool will require a source to act on. 
- Finally, we have the `Toolset`, that allows you to define groups of tools that you want to be able to load together. This can be useful for defining different groups based on agent or application. 

Since yaml is (mostly) a superset of JSON (i.e. every valid JSON file is also a valid YAML file), we can choose to define the file in traditional yaml format or JSON. To avoid issues with whitespace and indentation when defining multi-line SQL statements, we'll use JSON in this example.

> NOTE: The variable ALLOYDB_PASSWORD will be provided to MCP Toolbox as an environment variable defined in Cloud Run when we deploy it.

In [ ]:
# Reference: https://googleapis.github.io/genai-toolbox/resources/sources/spanner/
#            https://googleapis.github.io/genai-toolbox/resources/tools/
#            https://googleapis.github.io/genai-toolbox/resources/tools/spanner-sql/
#            https://googleapis.github.io/genai-toolbox/resources/sources/alloydb-pg/
#            https://googleapis.github.io/genai-toolbox/resources/tools/postgres-sql/

import os
import json

tools_config = {
  "sources": {
    "spanner-finance-graph-source": {
      "kind": "spanner",
      "project": f"{project_id}",
      "instance": f"{spanner_instance}",
      "database": f"{spanner_database}",
      "dialect": "googlesql"
    },
    "alloydb-finance-source": {
      "kind": "alloydb-postgres",
      "project": f"{project_id}",
      "region": f"{region}",
      "cluster": f"{alloydb_cluster}",
      "instance": f"{alloydb_instance}",
      "database": f"{alloydb_database}",
      "user": "postgres",
      "password": "${ALLOYDB_PASSWORD}",
      "ipType": "private"
    }
  },
  "tools": {
    "get_account_transfers": {
      "kind": "spanner-sql",
      "source": "spanner-finance-graph-source",
      "description": "Use this tool to get information about the 10 most recent account transfers.",
      "statement": "SELECT * FROM AccountTransferAccount LIMIT 10;"
    },
    "get_transactions": {
      "kind": "postgres-sql",
      "source": "alloydb-finance-source",
      "description": "Use this tool to look up information about the 10 most recent credit card transactions.",
      "statement": "SELECT * FROM transactions LIMIT 10;"
    }
  },
  "toolsets": {
    "finance-toolset": [
      "get_account_transfers",
      "get_transactions"
    ]
  }
}

with open("tools.yaml", "w") as file:
    file.write(json.dumps(tools_config))


### Write `tools.yaml` to Secret Manager

When deploying MCP Toolbox to Cloud Run, it is a best practice to write your tools.yaml file to a Secret Manager secret that can then be retrieved at runtime by Cloud Run. 

In [ ]:
# Create the secret
! gcloud secrets create tools --data-file=tools.yaml || gcloud secrets versions add tools --data-file=tools.yaml

In [ ]:
# Clean up the local file
import os
os.remove('tools.yaml')

### Write AlloyDB Password to Secret Manager

We can securely store the database password in Secret Manager to be accessed by Cloud Run as well.

In [ ]:
# Create the secret
! echo -n {alloydb_password} | gcloud secrets create alloydb-password --data-file=-


### Deploy MCP Toolbox to Cloud Run

> NOTE: We use a dedicated service account called toolbox-identity to assign the minimal required permissions for the MCP Toolbox service to access Spanner, AlloyDB, and Cloud SQL. The service account was already created via Terraform automation.

In [ ]:
# Reference: https://cloud.google.com/run/docs/authenticating/public#gcloud

# Define Toolbox Container Image
image = 'us-central1-docker.pkg.dev/database-toolbox/toolbox/toolbox:latest'

# Deploy to Cloud Run
! gcloud run deploy toolbox --no-user-output-enabled \
    --image={image} \
    --network={vpc} \
    --subnet={vpc} \
    --network-tags=toolbox \
    --service-account=toolbox-identity \
    --region={region} \
    --no-allow-unauthenticated \
    --set-secrets="/app/tools.yaml=tools:latest,ALLOYDB_PASSWORD=alloydb-password:latest" \
    --args="--tools-file=/app/tools.yaml","--address=0.0.0.0","--port=8080" \
    --vpc-egress=all-traffic \
    --ingress=internal \
    --min=1


## Test Tools

Tools are generally imported to an Agent, which chooses the correct tool for a given request and uses them to retrieve context and take other actions to achieve a state objective. In this step, we'll use the toolbox_core library to manually invoke the tools without an Agent framework to ensure they are working as expected. This pattern can be useful for testing during development, as well as running integration tests as part of a CI/CD pipeline when you make changes to tools. 

### Get the Toolbox Endpoint

In [ ]:
toolbox_url = ! gcloud run services describe toolbox --region {region} --format 'value(metadata.annotations."run.googleapis.com/urls")'
toolbox_url = json.loads(toolbox_url[0])[0]
print(toolbox_url)

### Get Auth Token

We required authenticate invocations of the Cloud Run service, so we first need to grab an auth token to use with our ToolboxClient. 

In [ ]:
import urllib

import google.auth.transport.requests
import google.oauth2.id_token


def get_auth_token(endpoint):
    # Cloud Run uses your service's hostname as the `audience` value
    # audience = 'https://my-cloud-run-service.run.app/'
    # For Cloud Run, `endpoint` is the URL (hostname + path) receiving the request
    # endpoint = 'https://my-cloud-run-service.run.app/my/awesome/url'
    
    auth_req = google.auth.transport.requests.Request()
    id_token = google.oauth2.id_token.fetch_id_token(auth_req, endpoint)

    return id_token

auth_token = get_auth_token(toolbox_url)

### Execute Tools

In [ ]:
import json
from toolbox_core import ToolboxClient, auth_methods

auth_token_provider = auth_methods.aget_google_id_token # can also use sync method

async with ToolboxClient(
    toolbox_url,
    client_headers={"Authorization": f"Bearer {auth_token}"},
) as client:
    tools = await client.load_toolset("finance-toolset")
    for t in tools:
        print(f"\nRunning tool: {t._ToolboxTool__url}")
        result = await t()
        json_result = json.loads(result)
        print("Tool result:\n")
        print(json.dumps(json_result, indent=2))


## Update Tools

So far, we have two simple database tools that run static SQL statements. This is useful for wiring up and testing the environment, but they won't be very useful for a real-world use case. To make our tools more dynamic, we'll start by adding parameters to our SQL queries, which allows the Agent to lookup data for specific customers based on their ID. 

We will use parameterized queries in our tools to prevent SQL injections. Query parameters can be used as substitutes for arbitrary expressions. Parameters cannot be used as substitutes for identifiers, column names, table names, or other parts of the query. Basic parameters types include string, integer, float, boolean types.

Notice that AlloyDB and Spanner use a difference syntax for parameter substitution (i.e. `$1`, `$2`, etc. for AlloyDB and `@param_name1`, `@param_name2`, etc. for Spanner). Details on the specific syntax for each tool type can be found [in the docs](https://googleapis.github.io/genai-toolbox/resources/tools/).

### Update the `tools.yaml` File

In [ ]:
# Reference: https://googleapis.github.io/genai-toolbox/resources/sources/spanner/
#            https://googleapis.github.io/genai-toolbox/resources/tools/
#            https://googleapis.github.io/genai-toolbox/resources/tools/spanner-sql/
#            https://googleapis.github.io/genai-toolbox/resources/sources/alloydb-pg/
#            https://googleapis.github.io/genai-toolbox/resources/tools/postgres-sql/

import os
import json

tools_config = {
  "sources": {
    "spanner-finance-graph-source": {
      "kind": "spanner",
      "project": f"{project_id}",
      "instance": f"{spanner_instance}",
      "database": f"{spanner_database}",
      "dialect": "googlesql"
    },
    "alloydb-finance-source": {
      "kind": "alloydb-postgres",
      "project": f"{project_id}",
      "region": f"{region}",
      "cluster": f"{alloydb_cluster}",
      "instance": f"{alloydb_instance}",
      "database": f"{alloydb_database}",
      "user": "postgres",
      "password": "${ALLOYDB_PASSWORD}",
      "ipType": "private"
    }
  },
  "tools": {
    "get_account_transfers": {
      "kind": "spanner-sql",
      "source": "spanner-finance-graph-source",
      "description": "Use this tool to get information about the 10 most recent account transfers.",
      "statement": """
        SELECT p.id AS customer_id, *
        FROM Person p
          JOIN PersonOwnAccount poa ON poa.id = p.id
          JOIN Account a ON a.id = poa.account_id
          FULL JOIN AccountTransferAccount ata ON (ata.id = poa.account_id OR ata.to_id = poa.account_id)
        WHERE p.id = @customer_id;
        """,
      "parameters": [
        {
          "name": "customer_id",
          "type": "integer",
          "description": "Unique customer id number"
        }
      ]
    },
    "get_transactions": {
      "kind": "postgres-sql",
      "source": "alloydb-finance-source",
      "description": "Use this tool to look up information about the 10 most recent credit card transactions.",
      "statement": """
        SELECT u.id AS customer_id, * 
        FROM users u
        JOIN transactions t ON t.client_id = u.id
        WHERE u.id = $1
        ORDER BY date DESC
        LIMIT 10
        """,
      "parameters": [
        {
          "name": "customer_id",
          "type": "integer",
          "description": "Unique customer id number"
        }
      ]
    }
  },
  "toolsets": {
    "finance-toolset": [
      "get_account_transfers",
      "get_transactions"
    ]
  }
}

with open("tools.yaml", "w") as file:
    file.write(json.dumps(tools_config))


### Write Updated `tools.yaml` to Secret Manager

In [ ]:
# Create the secret
! gcloud secrets versions add tools --data-file=tools.yaml

In [ ]:
# Clean up the local file
import os
os.remove('tools.yaml')

### Update Toolbox with New `tools.yaml` File

In [ ]:
# Reference: https://cloud.google.com/sdk/gcloud/reference/run/services/update

! gcloud run services update toolbox --no-user-output-enabled \
    --update-secrets="/app/tools.yaml=tools:latest,ALLOYDB_PASSWORD=alloydb-password:latest" \
    --region={region}
    

### Execute Tools

#### Run Without `customer_id`

We're first run the updated tools without providing a customer ID. Notice that this fails because this is a required parameter. 

In [ ]:
import json
from toolbox_core import ToolboxClient, auth_methods

# Get endpoint
toolbox_url = ! gcloud run services describe toolbox --region {region} --format 'value(metadata.annotations."run.googleapis.com/urls")'
toolbox_url = json.loads(toolbox_url[0])[0]
print(f"Toolbox Cloud Run endpoint: {toolbox_url}")

# Refresh auth_token
auth_token = get_auth_token(toolbox_url)
auth_token_provider = auth_methods.aget_google_id_token # can also use sync method

# Run tools 
async with ToolboxClient(
    toolbox_url,
    client_headers={"Authorization": f"Bearer {auth_token}"},
) as client:
    tools = await client.load_toolset("finance-toolset")
    for t in tools:
        print(f"\nRunning tool: {t._ToolboxTool__url}")
        result = await t()
        json_result = json.loads(result)
        print("Tool result:\n")
        print(json.dumps(json_result, indent=2))


#### Run With `customer_id`

We'll update our code to provide the required parameter this time. When Agents execute tools, they will be aware of the required parameters and can populate them in their tool calls based on the session context. Try setting different values (between 1 and 200) for customer_id and observe that the tools now return  account and transaction information about specific customers.  

In [ ]:
import json
from toolbox_core import ToolboxClient, auth_methods

# Set customer_id 
customer_id=11

# Refresh auth_token
auth_token = get_auth_token(toolbox_url)
auth_token_provider = auth_methods.aget_google_id_token # can also use sync method

# Run tools 
async with ToolboxClient(
    toolbox_url,
    client_headers={"Authorization": f"Bearer {auth_token}"},
) as client:
    tools = await client.load_toolset("finance-toolset")
    for t in tools:
        print(f"\nRunning tool: {t._ToolboxTool__url}")
        result = await t(customer_id) # Call the tool with the customer_id parameter here
        json_result = json.loads(result)
        print("Tool result:\n")
        print(json.dumps(json_result, indent=2))
